In [4]:
!pip install pip install pennylane pennylane-datasets scikit-learn

ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install


In [5]:
import pennylane as qml

data = qml.data.load("other", name="plus-minus")[0]
print("Atributos disponíveis no dataset:", dir(data))
print("Campos disponíveis:", getattr(data, "fields", "Sem campos"))

Atributos disponíveis no dataset: ['img_test', 'img_train', 'labels_test', 'labels_train']
Campos disponíveis: {}


In [ ]:

import pennylane as qml
from pennylane import numpy as np
from sklearn.metrics import accuracy_score

[ds] = qml.data.load("plus-minus")

X_train, Y_train = ds.img_train, ds.labels_train
X_test, Y_test   = ds.img_test, ds.labels_test

# Cada imagem é 16x16 = 256 pixels -> precisa de 8 qubits (2^8 = 256)
n_qubits = 8
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface="autograd")
def circuit(image, params):
    qml.AmplitudeEmbedding(image.flatten(), wires=range(n_qubits), normalize=True)
    for i in range(n_qubits):
        qml.RX(params[i], wires=i)
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    return qml.expval(qml.PauliZ(0))

def predict(x, params):
    return circuit(x, params)

def loss_fn(params, X, y):
    preds = np.array([predict(x, params) for x in X])
    probs = (preds + 1) / 2   # [-1,1] -> [0,1]
    eps = 1e-7
    probs = np.clip(probs, eps, 1 - eps)
    return -np.mean(y * np.log(probs) + (1 - y) * np.log(1 - probs))

np.random.seed(0)
params = np.random.random(n_qubits, requires_grad=True)

opt = qml.GradientDescentOptimizer(stepsize=0.3)
epochs = 20

for epoch in range(1, epochs + 1):
    params, loss = opt.step_and_cost(lambda w: loss_fn(w, X_train, Y_train), params)

    if epoch % 5 == 0 or epoch == 1:
        preds = np.array([predict(x, params) for x in X_train])
        labels = ((preds + 1) / 2 >= 0.5).astype(int)
        acc = accuracy_score(Y_train, labels)
        print(f"Epoch {epoch:02d} | Loss = {loss:.4f} | Train Acc = {acc:.3f}")

preds_test = np.array([predict(x, params) for x in X_test])
labels_test = ((preds_test + 1) / 2 >= 0.5).astype(int)
test_acc = accuracy_score(Y_test, labels_test)

print(f"\nAcurácia final no teste: {test_acc:.3f}")


Epoch 01 | Loss = 0.6828 | Train Acc = 0.238
Epoch 05 | Loss = 0.6795 | Train Acc = 0.238
Epoch 10 | Loss = 0.6764 | Train Acc = 0.243
Epoch 15 | Loss = 0.6743 | Train Acc = 0.237
Epoch 20 | Loss = 0.6729 | Train Acc = 0.239

Acurácia final no teste: 0.195


: 

In [9]:
!pip install qiskit qiskit-aer qiskit-machine-learning scikit-learn matplotlib numpy
!pip install qiskit-machine-learning


In [1]:
# INSTALAR DEPENDÊNCIAS SE PRECISAR:
# pip install --upgrade qiskit qiskit-aer qiskit-machine-learning pennylane scikit-learn matplotlib numpy

import random
import numpy as np
import pennylane as qml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report

from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit.primitives import Sampler
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_machine_learning.utils.loss_functions import CrossEntropyLoss
from qiskit_algorithms.optimizers import COBYLA

from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

try:
    from qiskit_algorithms.utils import algorithm_globals
    algorithm_globals.random_seed = SEED
except ImportError:
    pass

noise_model = NoiseModel()
error_1q = depolarizing_error(0.01, 1)
error_2q = depolarizing_error(0.02, 2)
noise_model.add_all_qubit_quantum_error(error_1q, ['h','x','y','z','rx','ry','rz','s','sdg','t','tdg'])
noise_model.add_all_qubit_quantum_error(error_2q, ['cx','cz','swap'])

simulator = AerSimulator(noise_model=noise_model, shots=1024, method='statevector')

[ds] = qml.data.load("plus-minus")

X = ds.img_train
y = ds.labels_train
X_test = ds.img_test
y_test = ds.labels_test

X = X.reshape((X.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))
scaler = MinMaxScaler((0,1))
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

num_classes = len(np.unique(y))
num_qubits = X.shape[1]

feature_map = ZZFeatureMap(num_qubits, reps=1)
ansatz = RealAmplitudes(num_qubits, reps=1)
qc = QuantumCircuit(num_qubits)
qc.compose(feature_map, inplace=True)
qc.compose(ansatz, inplace=True)

def interpret(x):
    return int(x, 2) % num_classes

sampler = Sampler(options={"backend": simulator})

qnn = SamplerQNN(
    circuit=qc,
    sampler=sampler,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    interpret=interpret,
    output_shape=num_classes,
)

def enviar_circuito_por_epoca(circuito, epoch):
    print(f"[Epoch {epoch+1}] Enviando circuito parametrizado para rede...")

    print(circuito)

vqc = NeuralNetworkClassifier(
    neural_network=qnn,
    optimizer=COBYLA(maxiter=1),  # rodar uma iteração por "epoch"
    loss=CrossEntropyLoss(),
    one_hot=True,
    callback=lambda weights, loss, step: enviar_circuito_por_epoca(
        qc.assign_parameters(weights), step
    ),
)

EPOCHS = 10
print("\nTREINANDO QNN MULTI-CLASS COM BACKEND RUIDOSO...")

for epoch in range(EPOCHS):
    vqc.fit(X_train, y_train)

    y_pred_val = vqc.predict(X_val)
    acc_val = accuracy_score(y_val, y_pred_val)
    print(f"[Epoch {epoch+1}] Accuracy validação: {acc_val:.4f}")

y_pred_test = vqc.predict(X_test)
print("\nRESULTADOS FINAIS NO TESTE:")
print("Acurácia:", accuracy_score(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))


/tmp/ipykernel_26400/68314634.py:89: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler(options={"backend": simulator})
/tmp/ipykernel_26400/68314634.py:91: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = SamplerQNN(



TREINANDO QNN MULTI-CLASS COM BACKEND RUIDOSO...


: 